In [ ]:
# Import statements
import numpy as np
import matplotlib.pyplot as pt
import glob
import os

from ase.io import read
from pyiron import Project, ase_to_pyiron

from molmod.units import *
from molmod.constants import *

from collections import namedtuple
from dataclasses import dataclass # replaces namedtuple with mutable attributes
%matplotlib inline


### Functions

In [ ]:
# Gather structures 

'''@dataclass
class s_object:
    structure: object
    ffatypes: object
    ffatype_ids: object
    bonds : object'''
        
        
class Sobject(object):
    def __init__(self,structure,ffatypes,ffatype_ids,bonds):
        self.structure = structure
        self.ffatypes = ffatypes
        self.ffatype_ids = ffatype_ids
        self.bonds = bonds
        
    

### Execution

In [ ]:
structures_database = {}

for block in glob.glob('./input_files/PXRD/Structures_Database/*/*.chk'):
    block_name = block.split('/')[-2]
    print(block_name)
    
    tmp = pr.create_job(pr.job_type.Yaff,'tmp',delete_existing_job=True)
    tmp.load_chk(block)
    structures_database[block_name] = Sobject(tmp.structure,tmp.ffatypes,tmp.ffatype_ids,tmp.bonds)

In [ ]:
pr_database = Project('PXRD/Juul_database')
pr_database_uff = Project('PXRD/Juul_database_uff')

#### Optimizations

In [ ]:
# yaff job function
def yaff_opt_job(pr, name, sobject, ffpars):
    job = pr.create_job(pr.job_type.Yaff, name, delete_existing_job=True)
    
    job.calc_minimize(max_iter=10000, cell=True)

    job.set_ffpars(fnames=ffpars)
    job.structure = sobject.structure
    job.ffatypes = sobject.ffatypes
    job.ffatype_ids = sobject.ffatype_ids
    job.bonds = sobject.bonds
    
    job.input['rcut'] = 11.0*angstrom
    job.input['alpha_scale'] = 2.86
    job.input['gcut_scale'] = 1.0
    job.input['tailcorrections'] = True

    job.executable.version = '2020'
    job.server.queue = 'slaking'
    job.server.cores = 1
    job.server.run_time = 5*60*60 # in seconds

    job.run()

In [ ]:
for bn,s in structures_database.items():
    ffpars = os.path.join('./input_files/PXRD/Structures_Database', bn, 'pars_cluster.txt')
    bn = bn.replace('-','_')
    yaff_opt_job(pr_database, bn, s, ffpars)

In [ ]:
for bn,s in structures_database.items():
    ffpars = os.path.join('./input_files/PXRD/Structures_Database', bn, 'pars_uff.txt')
    bn = bn.replace('-','_')
    yaff_opt_job(pr_database_uff, bn, s, ffpars)

#### Molecular Dynamics

In [ ]:
pr_database_md = Project('PXRD/Juul_database_md')
pr_database_uff_md = Project('PXRD/Juul_database_uff_md')

In [ ]:
def md_job_v2(pr,name,sobject,structure,ffpars,temp=300*kelvin,press=1e5*pascal,nsteps=400000, 
            time_step=0.5*femtosecond,n_print=200,timecon_thermo=100.0*femtosecond,timecon_baro=1000.0*femtosecond, repeat=(1,1,1), log_lammps=False):
    # MD code
    job = pr.create_job(pr.job_type.Yaff, name, delete_existing_job=True)
    job.calc_md(temperature=temp, pressure=press, nsteps=nsteps, time_step=time_step, n_print=n_print,
                timecon_thermo=timecon_thermo, timecon_baro=timecon_baro)
    
    job.set_ffpars(ffpars)
    
    job.input['rcut'] = 11.0*angstrom
    job.input['alpha_scale'] = 2.86
    job.input['gcut_scale'] = 1.0
    job.input['tailcorrections'] = True
    
    if log_lammps:
        job.input['log_lammps'] = 'lammps.log'
    
    # Load structure and atomtypes
    job.structure = structure.repeat(repeat)
    job.ffatypes = sobject.ffatypes
    job.ffatype_ids = np.tile(sobject.ffatype_ids,np.prod(repeat))
    
    # bonds will automatically be detected in the supercell
    # assume that optimization has taken care of structural irregularities
        
    job.enable_lammps(executable='2020_lammps')
    
    job.server.queue = 'slaking'
    job.server.cores = 8 
    job.server.run_time = 72*60*60 # in seconds

    job.run()
    

In [ ]:
for bn,s in structures_database.items():
    ffpars = os.path.join('./input_files/PXRD/Structures_Database', bn, 'pars_cluster.txt')
    bn = bn.replace('-','_')
    opt_structure = pr_database.load(bn).get_structure()

    # if the structure is 2D make a 1x1x5 supercell, if it is very small, use a 2x2x5 supercell
    if any(bn.startswith(k) for k in ['hcb','kgm','sql']):
        if len(opt_structure)<280: # 140 atoms per layer
            repeat = (2,2,5)
        else:
            repeat = (1,1,5)
    else:
        repeat = (1,1,1)

    md_job_v2(pr_database_md,bn,s,opt_structure,ffpars,repeat=repeat)

In [ ]:
for bn,s in structures_database.items():
    ffpars = os.path.join('./input_files/PXRD/Structures_Database', bn, 'pars_uff.txt')
    bn = bn.replace('-','_')
    opt_structure = pr_database_uff.load(bn).get_structure()

    # if the structure is 2D make a 1x1x5 supercell, if it is very small, use a 2x2x5 supercell
    if any(bn.startswith(k) for k in ['hcb','kgm','sql']):
        if len(opt_structure)<280: # 140 atoms per layer
            repeat = (2,2,5)
        else:
            repeat = (1,1,5)
    else:
        repeat = (1,1,1)

    md_job_v2(pr_database_uff_md,bn,s,opt_structure,ffpars,repeat=repeat)
        

#### PXRD

##### Static

In [ ]:
pr_database_pxrd = pr_database.create_group('PXRD')
pr_database_uff_pxrd = pr_database_uff.create_group('PXRD')

In [ ]:
def static_PXRD_job(pr,name,structure,refpattern=None):
    job = pr.create_job(pr.job_type.GPXRD, name, delete_existing_job=True)
    job.structure = structure
    job.input['jobtype'] = 'static'
    
    if refpattern is not None:
        job.set_reference_pattern(refpattern)

    job.server.queue = 'slaking'
    job.server.cores = 1 
    job.server.run_time = 15*60 # in seconds

    job.run()
    #return job

In [ ]:
for bn in structures_database.keys():
    name =  bn.replace('-','_')
    refpattern = glob.glob('./input_files/PXRD/Structures_Database/'+bn+'/*.dat')[0]
    job = pr_database.load(name)
    job_uff = pr_database_uff.load(name)

    static_PXRD_job(pr_database_pxrd,name,job.get_structure(),refpattern=refpattern)
    static_PXRD_job(pr_database_uff_pxrd,name,job_uff.get_structure(),refpattern=refpattern)

In [ ]:
from matplotlib.ticker import MaxNLocator

for bn in structures_database.keys():
    name =  bn.replace('-','_')
    
    job = pr_database_pxrd.load(name)
    job_uff = pr_database_uff_pxrd.load(name)
    
    ttheta_calc = job.get("output/ttheta_calc")
    int_calc = job.get("output/int_calc")
    
    ttheta_uff_calc = job_uff.get("output/ttheta_calc")
    int_uff_calc = job_uff.get("output/int_calc")

    ttheta_ref = job.get("output/ttheta_ref")
    int_ref = job.get("output/int_ref")
    
    stat_res = job.compare(np.array([ttheta_calc,int_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    stat_res_uff = job.compare(np.array([ttheta_uff_calc,int_uff_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    
    print(name)
    pt.figure()
    pt.plot(ttheta_calc,    (int_calc-int_calc.min())*stat_res['scalefactor'],lw=1,label='QuickFF, {:3.2f}'.format(stat_res['R_wp']))
    pt.plot(ttheta_uff_calc,(int_uff_calc-int_uff_calc.min())*stat_res_uff['scalefactor'],lw=1,label='UFF, {:3.2f}'.format(stat_res_uff['R_wp']))
    pt.plot(ttheta_ref,     (int_ref-int_ref.min()),lw=1,label='reference')
    
    ax1 = pt.gca()
    ax1.set_xlabel('2θ (°)')
    ax1.set_ylabel('Intensity (a.u.)')

    ax1.tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,      # ticks along the left edge are off
        right=False,         # ticks along the right edge are off
        labelleft=False) # labels along the left edge are off

    ax1.ticklabel_format(axis='y', style='plain', useOffset=False) # avoid scientific notation and offsets
    ax1.legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
        
    pt.show()

##### Background filtering

In [ ]:
from matplotlib.ticker import MaxNLocator
def remove_background(reference,locs=None,bkg_range=None,bkg_points=10,uniform=True,plot=False,fname=None,skiprows=0):
    """
        Remove the background the provided reference pattern

        ***Args***
        reference
            file name of reference data (file with two columns, no header)
            or a 2D array with (ttheta,intensity)
        skiprows
            number of rows to skip in reading file (when reference is a filename)
        locs
            node locations in degrees, default linspace in range with bkg_points
        bkg_range
            range for background points, default full ttheta range of reference pattern
        bkg_points
            set number of background points in fit
        uniform
            if false, the grid points are locally optimized towards the minima for a better interpolation
        plot
            plot the analysis and difference plot
        fname
            file location for the plot

        ***Returns***
            a new reference pattern (ttheta,intensity) array
    """

    # Interactively use the pyobjcryst code here, it should not be necessary to run a separate job for this
    try:
        import pyobjcryst
    except ImportError:
        raise ImportError("The pyobjcryst package is required for this")

    pp = pyobjcryst.powderpattern.PowderPattern()

    # Add the reference data
    if isinstance(reference,str) and os.path.exists(reference):
        pp.ImportPowderPattern2ThetaObs(reference,skiprows) # skip no lines
    elif isinstance(reference,np.ndarray):
        pp.SetPowderPatternX(reference[:,0]*deg)
        pp.SetPowderPatternObs(reference[:,1])
    ttheta = pp.GetPowderPatternX()/deg
    reference = pp.GetPowderPatternObs()

    if ttheta.shape == (0,):
        raise ValueError('Did not succeed in loading reference data. Try explicitely loading your data as an array instead.')


    # Background
    if locs is not None:
        bx = np.array(locs)
        # Keep first location index for clear plot
    else:
        if bkg_range is not None:
            assert len(bkg_range)==2
            bx=np.linspace(bkg_range[0],bkg_range[1],bkg_points)
        else:
            bx=np.linspace(ttheta.min(),ttheta.max(),bkg_points)
    
    # Keep first location index for clear plot
    idx = [np.argmin(np.abs(ttheta-bxi)) for bxi in bx]
    reference_idx = idx[0]
            
    # Adapt bx to minima of reference pattern in each neighbourhood (optional)
    if locs is None and not uniform:
        idx = [np.argmin(np.abs(ttheta-bxi)) for bxi in bx]
        step = (idx[1] - idx[0])//4
        for n in range(len(bx)):
            mn = -step if idx[n]>step else 0
            mx = step if n<(len(idx)-1) else 0
            bx[n] = ttheta[idx[n]+ mn + np.argmin(reference[idx[n]+mn:idx[n]+mx])]
            if n==0: reference_idx = idx[n]+ mn + np.argmin(reference[idx[n]+mn:idx[n]+mx])

    
    bx*=deg
    by=np.zeros(bx.shape)
    b=pp.AddPowderPatternBackground()
    b.SetInterpPoints(bx,by)
    b.UnFixAllPar()
    b.OptimizeBayesianBackground()

    no_bg = pp.GetPowderPatternObs()-pp.GetPowderPatternCalc()
    no_bg -= np.min(no_bg)
    

    # Plot the difference
    if plot:
        # Consider the difference for the delta plot
        height = no_bg.max()-no_bg.min()

        fig = pt.figure(figsize=(10,6))
        ax1 = pt.gca()
        background = pp.GetPowderPatternCalc() - pp.GetPowderPatternCalc().min()
        reference_pattern = reference - reference.min()
        ax1.plot(ttheta,background - (background[reference_idx]- reference_pattern[reference_idx]),lw=1,label='background')
        ax1.plot(ttheta,reference_pattern,lw=1,label='reference')
        ax1.plot(ttheta,no_bg-height*0.1,color='g',lw=1,label=r'$\Delta$')
        ax1.set_xlabel('2θ (°)')
        ax1.set_ylabel('Intensity (a.u.)')

        # Format the plot
        lims = pt.xlim()
        ax1.hlines(0,lims[0],lims[1],lw=0.1)
        ax1.set_xlim(lims)

        lims = pt.ylim()
        ax1.vlines(bx/deg,lims[0],lims[1],lw=0.1)

        ax1.tick_params(
            axis='y',          # changes apply to the y-axis
            which='both',      # both major and minor ticks are affected
            left=False,      # ticks along the left edge are off
            right=False,         # ticks along the right edge are off
            labelleft=False) # labels along the left edge are off

        ax1.ticklabel_format(axis='y', style='plain', useOffset=False) # avoid scientific notation and offsets

        ax1.legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
        ax1.xaxis.set_major_locator(MaxNLocator(integer=True))

        if fname is None:
            fig.tight_layout()
            pt.show()
        else:
            pt.savefig(fname+'.pdf',bbox_inches='tight')
        pt.close()

    if fname is not None:
        with open(fname + '.tsv','w') as f:
            for i in range(len(ttheta)):
                f.write("{:4.3f}\t{:10.8f}\n".format(ttheta[i],no_bg[i]))

    return np.array([ttheta, no_bg]).T

In [ ]:
# SEQUENCE BELOW
'dia_30-03-12_06-12-12',
'kgm_31-03-04_01-02-04',
'hcb_34-11-10_01-06-10',
'ctn_33-11-02_29-01-02_None',
'bor_18-08-01_28-01-01_None',
'hcb_11-02-06_None',
'sql_24-01-01_10-08-01',

In [ ]:
# 'dia_30-03-12_06-12-12'
bn = list(structures_database.keys())[0]
name =  bn.replace('-','_')
job = pr_database_pxrd.load(name)
refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T

refpattern_1 = remove_background(refpattern, bkg_range=(2,10), bkg_points=2, uniform=True,plot=False)
refpattern_2 = remove_background(refpattern_1, bkg_points=3, uniform=True,plot=False)
refpattern_3 = remove_background(refpattern_2, bkg_range=(20,40), bkg_points=3, uniform=True,plot=False,fname='./input_files/PXRD/Structures_Database/'+bn+'/background_filtered')
#job.compare(refpattern,refpattern_3,scale=False,plot=True)

In [ ]:
# 'kgm_31-03-04_01-02-04'
bn = list(structures_database.keys())[1]
name =  bn.replace('-','_')
print(name)
job = pr_database_pxrd.load(name)
#refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T

#refpattern_1 = remove_background(refpattern,    bkg_range=(1,8), bkg_points=4, uniform=False,plot=True)
#refpattern_2 = remove_background(refpattern_1, bkg_points=3, uniform=True,plot=False)
#refpattern_3 = remove_background(refpattern_2, bkg_range=(20,40), bkg_points=3, uniform=True,plot=False,fname='./input_files/PXRD/Structures_Database/'+bn+'/background_filtered')
#job.compare(refpattern,refpattern_3,scale=False,plot=True)

In [ ]:
# 'hcb_34-11-10_01-06-10'
bn = list(structures_database.keys())[2]
name =  bn.replace('-','_')
job = pr_database_pxrd.load(name)
refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T
plot=False

refpattern_1 = remove_background(refpattern,   bkg_points=5, uniform=True,plot=plot)
refpattern_2 = remove_background(refpattern_1, bkg_range=(10,40), bkg_points=2, uniform=True,plot=plot)
refpattern_3 = remove_background(refpattern_2, bkg_points=5, uniform=True,plot=plot)
refpattern_4 = remove_background(refpattern_3, bkg_range=(10,40), bkg_points=3, uniform=True,plot=plot)
refpattern_5 = remove_background(refpattern_4, bkg_range=(35,40), bkg_points=3, uniform=True,plot=plot,fname='./input_files/PXRD/Structures_Database/'+bn+'/background_filtered')
#job.compare(refpattern,refpattern_5,scale=False,plot=True)

In [ ]:
# 'ctn_33-11-02_29-01-02_None'
bn = list(structures_database.keys())[3]
name =  bn.replace('-','_')
job = pr_database_pxrd.load(name)
refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T

plot=True
# no fit necessary
#job.compare(refpattern,refpattern_5,scale=False,plot=True)

In [ ]:
# 'bor_18-08-01_28-01-01_None'
bn = list(structures_database.keys())[4]
name =  bn.replace('-','_')
job = pr_database_pxrd.load(name)
refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T

plot=False

refpattern_1 = remove_background(refpattern, bkg_points=3, uniform=True,plot=plot)
refpattern_2 = remove_background(refpattern_1, bkg_range=(9,35), bkg_points=4, uniform=True,plot=plot)
refpattern_3 = remove_background(refpattern_2,  bkg_points=4, uniform=True,plot=plot,fname='./input_files/PXRD/Structures_Database/'+bn+'/background_filtered')
#job.compare(refpattern,refpattern_3,scale=False,plot=True)

In [ ]:
# 'hcb_11-02-06_None'
bn = list(structures_database.keys())[5]
name =  bn.replace('-','_')
job = pr_database_pxrd.load(name)
refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T

plot=False

refpattern_1 = remove_background(refpattern, bkg_points=4, uniform=True,plot=plot)
refpattern_2 = remove_background(refpattern_1, bkg_points=2, uniform=True,plot=plot)
refpattern_3 = remove_background(refpattern_2,  bkg_points=4, uniform=True,plot=plot)
refpattern_4 = remove_background(refpattern_3, bkg_range=(16,60), bkg_points=2, uniform=True,plot=plot)
refpattern_5 = remove_background(refpattern_4, bkg_points=4, uniform=False,plot=plot)
refpattern_6 = remove_background(refpattern_5, bkg_range=(38,60), bkg_points=3, uniform=False,plot=plot,fname='./input_files/PXRD/Structures_Database/'+bn+'/background_filtered')

#job.compare(refpattern,refpattern_6,scale=False,plot=True)

In [ ]:
#
bn = list(structures_database.keys())[6]
name =  bn.replace('-','_')
job = pr_database_pxrd.load(name)
refpattern = np.array([job['output/ttheta_ref'],job['output/int_ref']]).T

plot=False

refpattern_1 = remove_background(refpattern,  bkg_range=(2,13), bkg_points=3, uniform=False,plot=plot)
refpattern_2 = remove_background(refpattern_1,bkg_range=(2,50), bkg_points=2, uniform=True,plot=plot)
refpattern_3 = remove_background(refpattern_2, bkg_points=4, uniform=True,plot=plot)
refpattern_4 = remove_background(refpattern_3, bkg_points=7, uniform=False,plot=plot)
refpattern_5 = remove_background(refpattern_4, bkg_range=(15,50), bkg_points=7, uniform=False,plot=plot)
refpattern_6 = remove_background(refpattern_5, bkg_points=4, uniform=False,plot=plot,fname='./input_files/PXRD/Structures_Database/'+bn+'/background_filtered')

#job.compare(refpattern,refpattern_6,scale=False,plot=True)

##### Static w background filtering

In [ ]:
pr_database_pxrd_bg = pr_database.create_group('PXRD_bg')
pr_database_uff_pxrd_bg = pr_database_uff.create_group('PXRD_bg')

In [ ]:
for bn in structures_database.keys():
    name =  bn.replace('-','_')
    background_filtering = glob.glob('./input_files/PXRD/Structures_Database/'+bn+'/background_filtered.tsv')
    if len(background_filtering)==0:
        refpattern = glob.glob('./input_files/PXRD/Structures_Database/'+bn+'/*.dat')[0]
    else:
        refpattern = background_filtering[0]
    job = pr_database.load(name)
    job_uff = pr_database_uff.load(name)

    static_PXRD_job(pr_database_pxrd_bg,name,job.get_structure(),refpattern=refpattern)
    static_PXRD_job(pr_database_uff_pxrd_bg,name,job_uff.get_structure(),refpattern=refpattern)

In [ ]:
from matplotlib.ticker import MaxNLocator

for bn in structures_database.keys():
    name =  bn.replace('-','_')
    
    job = pr_database_pxrd_bg.load(name)
    job_uff = pr_database_uff_pxrd_bg.load(name)
    
    ttheta_calc = job.get("output/ttheta_calc")
    int_calc = job.get("output/int_calc")
    
    ttheta_uff_calc = job_uff.get("output/ttheta_calc")
    int_uff_calc = job_uff.get("output/int_calc")

    ttheta_ref = job.get("output/ttheta_ref")
    int_ref = job.get("output/int_ref")
    
    stat_res = job.compare(np.array([ttheta_calc,int_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    stat_res_uff = job.compare(np.array([ttheta_uff_calc,int_uff_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    
    print(name)
    pt.figure()
    pt.plot(ttheta_calc,    (int_calc-int_calc.min())*stat_res['scalefactor'],lw=1,label='QuickFF, {:3.2f}'.format(stat_res['R_wp']))
    pt.plot(ttheta_uff_calc,(int_uff_calc-int_uff_calc.min())*stat_res_uff['scalefactor'],lw=1,label='UFF, {:3.2f}'.format(stat_res_uff['R_wp']))
    pt.plot(ttheta_ref,     (int_ref-int_ref.min()),lw=1,label='reference')
    
    ax1 = pt.gca()
    ax1.set_xlabel('2θ (°)')
    ax1.set_ylabel('Intensity (a.u.)')

    ax1.tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,      # ticks along the left edge are off
        right=False,         # ticks along the right edge are off
        labelleft=False) # labels along the left edge are off

    ax1.ticklabel_format(axis='y', style='plain', useOffset=False) # avoid scientific notation and offsets
    ax1.legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
        
    pt.show()

##### Dynamic

In [ ]:
pr_database_md_pxrd = pr_database_md.create_group('PXRD')
pr_database_uff_md_pxrd = pr_database_uff_md.create_group('PXRD')

In [ ]:
def dynamic_PXRD_job_v2(pr,name,md_job,refpattern=None,start=1000,num_frames=50,cores=1,cluster='victini',reservation_tag=None):
    job = pr.create_job(pr.job_type.GPXRD, name, delete_existing_job=True)
    job.load_trajectory(md_job,start=start,num_frames=num_frames)
    
    if refpattern is not None:
        job.set_reference_pattern(refpattern)
    
    # We dont want individual fhkl indices
    job.input['save_fhkl'] = False
    
    job.server.queue = cluster
    job.server.cores = cores
    job.server.run_time = 5*60*60 # in seconds
    job.server.reservation_tag = reservation_tag

    job.run()
    #return job

In [ ]:
for bn in structures_database.keys():
    name =  bn.replace('-','_')
    refpattern = glob.glob('./input_files/PXRD/Structures_Database/'+bn+'/*.dat')[0]
    job = pr_database_md.load(name)
    job_uff = pr_database_uff_md.load(name)

    if job.status.finished:
        dynamic_PXRD_job_v2(pr_database_md_pxrd,name,job,refpattern=refpattern,cluster='slaking',cores=1)
    if job_uff.status.finished:
        dynamic_PXRD_job_v2(pr_database_uff_md_pxrd,name,job_uff,refpattern=refpattern,cluster='slaking',cores=1)

In [ ]:
from matplotlib.ticker import MaxNLocator

for bn in structures_database.keys():
    name =  bn.replace('-','_')
    
    try:
        job = pr_database_md_pxrd.load(name)
        job_uff = pr_database_uff_md_pxrd.load(name)

        ttheta_calc = job.get("output/ttheta_calc")
        int_calc = job.get("output/int_calc")

        ttheta_uff_calc = job_uff.get("output/ttheta_calc")
        int_uff_calc = job_uff.get("output/int_calc")

        ttheta_ref = job.get("output/ttheta_ref")
        int_ref = job.get("output/int_ref")
    
    except AttributeError:
        continue
    
    stat_res = job.compare(np.array([ttheta_calc,int_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    stat_res_uff = job.compare(np.array([ttheta_uff_calc,int_uff_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    
    print(name)
    pt.figure(figsize=(20,10))
    pt.plot(ttheta_calc,    (int_calc-int_calc.min())*stat_res['scalefactor'],lw=1,label='QuickFF, {:3.2f}'.format(stat_res['R_wp']))
    pt.plot(ttheta_uff_calc,(int_uff_calc-int_uff_calc.min())*stat_res_uff['scalefactor'],lw=1,label='UFF, {:3.2f}'.format(stat_res_uff['R_wp']))
    pt.plot(ttheta_ref,     (int_ref-int_ref.min()),lw=1,label='reference')
    
    ax1 = pt.gca()
    ax1.set_xlabel('2θ (°)')
    ax1.set_ylabel('Intensity (a.u.)')

    ax1.tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,      # ticks along the left edge are off
        right=False,         # ticks along the right edge are off
        labelleft=False) # labels along the left edge are off

    ax1.ticklabel_format(axis='y', style='plain', useOffset=False) # avoid scientific notation and offsets
    ax1.legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
        
    pt.savefig(name+'.png',bbox_inches='tight')
    pt.show()

##### Dynamic w background filtering

In [ ]:
pr_database_md_pxrd_bg = pr_database_md.create_group('PXRD_bg')
pr_database_uff_md_pxrd_bg = pr_database_uff_md.create_group('PXRD_bg')

In [ ]:
for bn in structures_database.keys():
    name =  bn.replace('-','_')
    background_filtering = glob.glob('./input_files/PXRD/Structures_Database/'+bn+'/background_filtered.tsv')
    if len(background_filtering)==0:
        refpattern = glob.glob('./input_files/PXRD/Structures_Database/'+bn+'/*.dat')[0]
    else:
        refpattern = background_filtering[0]

    job = pr_database_md.load(name)
    job_uff = pr_database_uff_md.load(name)

    if job.status.finished:
        if pr_database_md_pxrd_bg.load(name) is None:
            dynamic_PXRD_job_v2(pr_database_md_pxrd_bg,name,job,refpattern=refpattern,cluster='slaking',cores=1)
        elif pr_database_md_pxrd_bg.load(name).status.aborted:
            dynamic_PXRD_job_v2(pr_database_md_pxrd_bg,name,job,refpattern=refpattern,cluster='slaking',cores=8)
    if job_uff.status.finished:
        if pr_database_uff_md_pxrd_bg.load(name) is None:
            dynamic_PXRD_job_v2(pr_database_uff_md_pxrd_bg,name,job_uff,refpattern=refpattern,cluster='slaking',cores=1)
        elif pr_database_uff_md_pxrd_bg.load(name).status.aborted:
            dynamic_PXRD_job_v2(pr_database_uff_md_pxrd_bg,name,job_uff,refpattern=refpattern,cluster='slaking',cores=8)

In [ ]:
from matplotlib.ticker import MaxNLocator

for bn in structures_database.keys():
    name =  bn.replace('-','_')
    
    try:
        job = pr_database_md_pxrd_bg.load(name)
        job_uff = pr_database_uff_md_pxrd_bg.load(name)

        ttheta_calc = job.get("output/ttheta_calc")
        int_calc = job.get("output/int_calc")

        ttheta_uff_calc = job_uff.get("output/ttheta_calc")
        int_uff_calc = job_uff.get("output/int_calc")

        ttheta_ref = job.get("output/ttheta_ref")
        int_ref = job.get("output/int_ref")
    
    except AttributeError:
        continue
    
    stat_res = job.compare(np.array([ttheta_calc,int_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    stat_res_uff = job.compare(np.array([ttheta_uff_calc,int_uff_calc]).T, np.array([ttheta_ref,int_ref]).T,scale='optimal',verbose=False,plot=False)
    
    print(name)
    pt.figure(figsize=(20,10))
    pt.plot(ttheta_calc,    (int_calc-int_calc.min())*stat_res['scalefactor'],lw=1,label='QuickFF, {:3.2f}'.format(stat_res['R_wp']))
    pt.plot(ttheta_uff_calc,(int_uff_calc-int_uff_calc.min())*stat_res_uff['scalefactor'],lw=1,label='UFF, {:3.2f}'.format(stat_res_uff['R_wp']))
    pt.plot(ttheta_ref,     (int_ref-int_ref.min()),lw=1,label='reference')
    
    ax1 = pt.gca()
    ax1.set_xlabel('2θ (°)')
    ax1.set_ylabel('Intensity (a.u.)')

    ax1.tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,      # ticks along the left edge are off
        right=False,         # ticks along the right edge are off
        labelleft=False) # labels along the left edge are off

    ax1.ticklabel_format(axis='y', style='plain', useOffset=False) # avoid scientific notation and offsets
    ax1.legend(bbox_to_anchor=(1.1,.5), loc='center left',frameon=False)
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
        
    #pt.savefig(name+'.png',bbox_inches='tight')
    pt.show()